In [1]:
import os
import cv2
import gc
import mediapipe as mp
import numpy as np

In [ ]:
def choose_best_face(detections, w, h, px=0.5, py=0.65):
    """
    To Select da face closest to da fokin priority point (px, py) in normalized coords.
    """
    best_det = None
    best_dist = float("inf")

    for det in detections:
        bbox = det.location_data.relative_bounding_box

        cx = (bbox.xmin + bbox.width / 2) * w
        cy = (bbox.ymin + bbox.height / 2) * h

        dx = cx - (px * w)
        dy = cy - (py * h)
        dist = dx * dx + dy * dy  

        if dist < best_dist:
            best_dist = dist
            best_det = det

    return best_det


In [ ]:
def extract_lip_frames(
    video_path,
    out_dir,
    start_time=0,
    end_time=None,
    priority_x=0.5,
    priority_y=0.65,
    verbose=True
):
    """
    Extract Bhigi hoth frames using MP(CPU),
    prioritizing the face nearest to the specified region px,py.
    """

    os.makedirs(out_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception(f"Cannot open video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps if fps > 0 else 0

    if verbose:
        print(f"🎥 Video: {fps:.2f} fps | {total_frames} frames | {duration:.2f}s")

    if end_time is None or end_time > duration:
        end_time = duration

    cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)

    frame_idx = 0
    detected = 0

    face_det = None

    try:
        face_det = mp.solutions.face_detection.FaceDetection(
            model_selection=1,
            min_detection_confidence=0.3
        )

        while True:
            current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
            if current_time > end_time:
                break

            ret, frame = cap.read()
            if not ret:
                break

            # Resize large frames for speed
            h, w = frame.shape[:2]
            if w > 960:
                scale = 960 / w
                frame = cv2.resize(frame, None, fx=scale, fy=scale)
                h, w = frame.shape[:2]

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_det.process(rgb)

            if results.detections:
                det = choose_best_face(
                    results.detections,
                    w, h,
                    px=priority_x,
                    py=priority_y
                )

                if det is not None:
                    bbox = det.location_data.relative_bounding_box

                    x1 = int(bbox.xmin * w)
                    y1 = int(bbox.ymin * h)
                    bw = int(bbox.width * w)
                    bh = int(bbox.height * h)

                    # Focus on lower face (mouth region)
                    y1 = y1 + int(bh * 0.4)
                    bh = int(bh * 0.6)

                    # Padding + clipping
                    pad = 10
                    x1 = max(x1 - pad, 0)
                    y1 = max(y1 - pad, 0)
                    x2 = min(x1 + bw + 2 * pad, w)
                    y2 = min(y1 + bh + 2 * pad, h)

                    crop = frame[y1:y2, x1:x2]

                    if crop.size > 0:
                        cv2.imwrite(
                            os.path.join(out_dir, f"{frame_idx:06d}.jpg"),
                            crop
                        )
                        detected += 1

            frame_idx += 1

            # garbage collection happening
            if frame_idx % 200 == 0:
                gc.collect()

    finally:
        if face_det is not None:
            face_det.close()
        cap.release()
        gc.collect()

    if verbose:
        print(f"🎬 Extracted {detected}/{frame_idx} frames")
        if detected == 0:
            print("⚠️ WARNING: No face detected in any frame")

    return detected > 0


In [ ]:
extract_lip_frames(
    video_path="C:/MajorProject/cropped_videos/Sample1.mp4",
    out_dir="lip_frames",
    start_time=0,
    end_time=10,
    priority_x=0.7,   # x cord from top left
    priority_y=0.3,  # y cord from top left
    verbose=True
)

🎥 Video: 29.97 fps | 2485 frames | 82.92s
🎬 Extracted 301/301 frames


True